## Topographic Complexity/Variability: Mexican Hat Wavelet Analysis  
* The original MATLAB code was developed from Dr. Adam M. Booth (Portland State Univeristy).  
    * Citations:  
        * Booth, A.M., Roering, J.J., Perron, J.T., 2009. Automated landslide mapping using spectral analysis and high-resolution topographic data: Puget Sound lowlands, Washington, and Portland    Hills, Oregon. Geomorphology 109, 132-147. https://doi.org/10.1016/j.geomorph.2009.02.027  
        * Booth, A.M., LaHusen, S.R., Duvall, A.R., Montgomery, D.R., 2017. Holocene history of deep-seated landsliding in the North Fork Stillaguamish River valley from surface roughness analysis, radiocarbon dating, and numerical landscape evolution modeling. Journal of Geophysical Research: Earth Surface 122, 456-472. https://doi.org/10.1002/2016JF003934  
* This MATLAB code was later adapted and revised by Dr. Sean R. LaHusen & Erich N. Herzig (Univeristy of Washington)
    * Citations:  
       * LaHusen, S.R., Duvall, A.R., Booth, A.M., Montgomery, D.R., 2016. Surface roughness dating of long-runout landslides near Oso, Washington (USA), reveals persistent postglacial hillslope instability. Geology 44, 111-114. https://doi.org/10.1130/G37267.1  
       * LaHusen, S.R., Duvall, A.R., Booth, A.M., Grant, A., Mishkin, B.A., Montgomery, D.R., Struble, W., Roering, J.J., Wartman, J., 2020. Rainfall triggers more deep-seated landslides than Cascadia earthquakes in the Oregon Coast Range, USA. Science Advances 6, eaba6790. https://doi.org/10.1126/sciadv.aba6790  
       * Herzig et al. (2023 in print) Bulletin of the Seismological Society of America. (details TBA)  
* In November, 2023; this is code translated and optimized into this python version by Dr. Larry Syu-Heng Lai (Univeristy of Washington)  
    * Citations: TBA  

### Initial setup

In [ ]:
import numpy as np
import scipy.signal
import rasterio

### Define data pathes

In [ ]:
# Define your file paths and file names separately
input_folder = '/Users/larryslai/Library/CloudStorage/Dropbox/QGIS/WA LiDAR/'
input_file_name = 'Test_DEM.tif'
#input_file_name = 'Tokeland_DEM.tif'
#input_file_name = 'Nemah_DEM.tif'
#input_file_name = 'Francies_DEM.tif'

output_folder = '/Users/larryslai/Library/CloudStorage/Dropbox/QGIS/WA LiDAR/'
output_file_name = 'Test_pymexhat.tif'
#output_file_name = 'Tokeland_pymexhat.tif'
#output_file_name = 'Nemah_pymexhat.tif'
#output_file_name = 'Francies_pymexhat.tif'

# Combine folder and file names to create the full paths
input_tif_path = input_folder + input_file_name
output_tif_path = output_folder + output_file_name

### Read DEM

In [ ]:
# Read the input GeoTIFF
with rasterio.open(input_tif_path) as src:
    dem = src.read(1)  # Read the first band into a 2D array
    meta = src.meta

See coordinate system info of the GeoTIFF

In [ ]:
# Open the GeoTIFF file
with rasterio.open(input_tif_path) as src:
    # Read the CRS
    crs = src.crs
    
    # Print the CRS information
    print(f"CRS: {crs}")
    print(f"CRS as WKT: {crs.wkt}")
    print(f"CRS as PROJ string: {crs.to_proj4()}")
    print(f"CRS as EPSG code: {crs.to_epsg()}")
    print(f"CRS as dictionary: {crs.to_dict()}")

### Mexican Hat Wavelet Analysis Function (original slower ver)

In [ ]:
def conv2_mexh(dem, a, dx):
    """
    Perform the 2D Continuous Wavelet Transform using the Mexican Hat wavelet.
    
    :param dem: Digital elevation model (2D numpy array).
    :param a: Wavelet scale.
    :param dx: Grid spacing.
    :return: Tuple of (C, frq, wave), the wavelet coefficients and frequencies.
    """
    # Generate the Mexican Hat wavelet kernel at wavelet scale a
    sz = int(np.ceil(8 * a))  # Kernel size
    X, Y = np.meshgrid(np.arange(-sz, sz+1), np.arange(-sz, sz+1))

    # Scaled Mexican Hat wavelet (psi)
    psi = (-1 / (np.pi * (a * dx)**4)) * (1 - (X**2 + Y**2) / (2 * a**2)) * np.exp(-(X**2 + Y**2) / (2 * a**2))

    # Convolve dem with psi
    C = dx**2 * scipy.signal.convolve2d(dem * 0.3048, psi, mode='same')

    # Mask edge effects with NaN values
    fringeval = int(np.ceil(a * 4))
    C[:fringeval, :] = np.nan
    C[-fringeval:, :] = np.nan
    C[:, :fringeval] = np.nan
    C[:, -fringeval:] = np.nan

    # Frequency and wavelength calculations
    wave = 2 * np.pi * dx * a / np.sqrt(5 / 2)  # Wavelength
    frq = 1 / wave  # Frequency
    
    return C, frq, wave

Set parameters

In [ ]:
a = 4.1  # Wavelet scale
dx = 1.8288  # Grid spacing

Perform the function

In [ ]:
C, frq, wave = conv2_mexh(dem, a, dx)

Output data into GeoTIFF

In [ ]:
# Define georeference system
coord_ref_sys_code = 32149 #NAD38 Washington South
# coord_ref_sys_code = 32610 #WGS84_UTM Zone 10N

# Prepare the metadata for writing the output GeoTIFF
meta.update({
    'dtype': 'float32',
    'nodata': np.nan,
    'crs': f'EPSG:{coord_ref_sys_code}'
})

# Write the result to a new GeoTIFF
with rasterio.open(output_tif_path, 'w', **meta) as dst:
    dst.write(C.astype(np.float32), 1)  # Write the computed C as the first band

### Mexican Hat Wavelet Analysis Function - Optimized with FFT convolution

In [ ]:
def conv2_mexh_fft(dem, a, dx):
    """
    Perform the 2D Continuous Wavelet Transform using the Mexican Hat wavelet.
    
    :param dem: Digital elevation model (2D numpy array).
    :param a: Wavelet scale.
    :param dx: Grid spacing.
    :return: Tuple of (C, frq, wave), the wavelet coefficients and frequencies.
    """
    # Kernel size, assuming the wavelet decays to 0 at the edges
    sz = int(np.ceil(8 * a))  
    X, Y = np.meshgrid(np.arange(-sz, sz+1), np.arange(-sz, sz+1))

    # Scaled Mexican Hat wavelet (psi)
    psi = (-1 / (np.pi * (a * dx)**4)) * (1 - (X**2 + Y**2) / (2 * a**2)) * np.exp(-(X**2 + Y**2) / (2 * a**2))

    # Convolve dem with psi using FFT for speed optimization
    C = scipy.signal.fftconvolve(dem, psi, mode='same')

    # Frequency and wavelength calculations
    wave = 2 * np.pi * dx * a / np.sqrt(5 / 2)  # Wavelength
    frq = 1 / wave  # Frequency

    return C, frq, wave

Set parameters  
'a' = aproximated mexican hat wavelength/4

In [ ]:
a = 4.1  # Wavelet scale
dx = 1.8288  # Grid spacing

##### Perform the wavelet transform on the entire dataset (for smaller DEM file)

In [ ]:
C_full, frq, wave = conv2_mexh_fft(dem, a, dx)

Output data into GeoTIFF (Optimzation made for faster writing)    
* Enabling geotiff compression to reduce writing time
* Enabling Tile-based writing if needed
* Enabling BIGTIFF parameter to allow writing a large GeoTIFF

In [ ]:
# Define georeference system
#coord_ref_sys_code = 32149 #NAD38 Washington South
#coord_ref_sys_code = 32610 #WGS84_UTM Zone 10N
#coord_ref_sys_code = crs.to_epsg() #Use the same georeference code as the original GeoTIFF file

# Prepare the metadata for writing the output GeoTIFF
meta.update({
    'dtype': 'float32',
    'nodata': np.nan,
    #'crs': f'EPSG:{coord_ref_sys_code}',
    'compress': 'lzw',  # Using LZW compression
    'tiled': True,      # Writing in tiles
    'BIGTIFF': 'IF_SAFER'    # Explicitly use BigTIFF format
})

# Write the result to a new GeoTIFF
with rasterio.open(output_tif_path, 'w', **meta) as dst:
    dst.write(C_full, 1)  # Write the computed C_full as the first band